## 範例重點
* 學習如何在 keras 中加入自定義的 callbacks
* 知道 callback function 可以動作的時機

In [1]:
import os
import keras

# 本範例不需使用 GPU, 將 GPU 設定為 "無"
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# 資料前處理 - X 標準化
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# 資料前處理 -Y 轉成 onehot
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
from keras.layers import BatchNormalization

"""
建立神經網路，並加入 BN layer
"""
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(input_layer)
            x = BatchNormalization()(x)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(x)
            x = BatchNormalization()(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [6]:
## 超參數設定
LEARNING_RATE = 1e-3
EPOCHS = 50
BATCH_SIZE = 1024
MOMENTUM = 0.95

In [7]:
"""
# 載入 Callbacks，撰寫一個 f1 score 的 callback function
"""

from keras.callbacks import Callback
from sklearn.metrics import f1_score

class f1sc(Callback):
    def on_train_begin(self, epoch, logs = {}):
        
        logs = logs or {}
        record_items = ["val_auc", "val_f1sc", "val_fp", "val_fn", "val_tp", "val_tn"]
        print(logs)
#         for i in record_items:
#             if i not in self.params['metrics']:
#                 self.params['metrics'].append(i)
    
    def on_epoch_end(self, epoch, logs = {}, thres=0.5):
        logs = logs or {}
        print(logs)
#         y_true = self.validation_data[1].argmax(axis = 1)
#         y_pred = self.model.predict(self.validation_data[0])
#         y_pred = (y_pred[:, 1] >= thres) * 1
        
#         logs["val_f1sc"] = f1_score(y_true = y_true, y_pred = y_pred, average="weighted")
        
log_f1sc = f1sc()

In [8]:
model = build_mlp(input_shape=x_train.shape[1:])
model.summary()
optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

model.fit(x_train, y_train, 
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          validation_data=(x_test, y_test), 
          shuffle=True,
          callbacks=[log_f1sc]
         )

# Collect results
train_loss = model.history.history["loss"]
valid_loss = model.history.history["val_loss"]
train_acc = model.history.history["accuracy"]
valid_acc = model.history.history["val_accuracy"]

# 在訓練後，將 f1sc 紀錄調出
valid_f1sc = model.history.history['val_f1sc']

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3072)]            0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
batch_normalization_2 (Batch (None, 128)              

Epoch 15/50
49/49 [==============================] - 5s 100ms/step - loss: 1.1756 - accuracy: 0.5922 - val_loss: 1.4496 - val_accuracy: 0.4880
Epoch 16/50
49/49 [==============================] - 5s 103ms/step - loss: 1.1526 - accuracy: 0.6026 - val_loss: 1.4512 - val_accuracy: 0.4925
Epoch 17/50
49/49 [==============================] - 5s 105ms/step - loss: 1.1296 - accuracy: 0.6110 - val_loss: 1.4606 - val_accuracy: 0.4899
Epoch 18/50
49/49 [==============================] - 5s 104ms/step - loss: 1.1089 - accuracy: 0.6181 - val_loss: 1.4490 - val_accuracy: 0.4935
Epoch 19/50
49/49 [==============================] - 5s 106ms/step - loss: 1.0868 - accuracy: 0.6264 - val_loss: 1.4510 - val_accuracy: 0.4910
Epoch 20/50
49/49 [==============================] - 5s 103ms/step - loss: 1.0668 - accuracy: 0.6347 - val_loss: 1.4584 - val_accuracy: 0.4922
Epoch 21/50
49/49 [==============================] - 5s 103ms/step - loss: 1.0466 - accuracy: 0.6427 - val_loss: 1.4438 - val_accuracy: 0.4944

49/49 [==============================] - 5s 100ms/step - loss: 0.7200 - accuracy: 0.7692 - val_loss: 1.5274 - val_accuracy: 0.4991
Epoch 39/50
49/49 [==============================] - 5s 100ms/step - loss: 0.7026 - accuracy: 0.7780 - val_loss: 1.5354 - val_accuracy: 0.5029
Epoch 40/50
49/49 [==============================] - 5s 97ms/step - loss: 0.6846 - accuracy: 0.7846 - val_loss: 1.5702 - val_accuracy: 0.4915
Epoch 41/50
49/49 [==============================] - 5s 96ms/step - loss: 0.6681 - accuracy: 0.7906 - val_loss: 1.5593 - val_accuracy: 0.4984
Epoch 42/50
49/49 [==============================] - 5s 105ms/step - loss: 0.6514 - accuracy: 0.7987 - val_loss: 1.5527 - val_accuracy: 0.4995
Epoch 43/50
49/49 [==============================] - 5s 99ms/step - loss: 0.6348 - accuracy: 0.8047 - val_loss: 1.5699 - val_accuracy: 0.4991
Epoch 44/50
49/49 [==============================] - 5s 101ms/step - loss: 0.6163 - accuracy: 0.8122 - val_loss: 1.5858 - val_accuracy: 0.4985
Epoch 45/50
49

KeyError: 'val_f1sc'

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(range(len(train_loss)), train_loss, label="train loss")
plt.plot(range(len(valid_loss)), valid_loss, label="valid loss")
plt.legend()
plt.title("Loss")
plt.show()

plt.plot(range(len(train_acc)), train_acc, label="train accuracy")
plt.plot(range(len(valid_acc)), valid_acc, label="valid accuracy")
plt.legend()
plt.title("Accuracy")
plt.show()

plt.plot(range(len(valid_f1sc)), valid_f1sc, label="valid f1-score")
plt.legend()
plt.title("F1-score")
plt.show()


## Work
1. 請嘗試寫一個 callback 用來記錄各類別在訓練過程中，對驗證集的 True Positive 與 True Negative